In [ ]:
# Colab upload/download shims
try:
    from google.colab import files  # noqa: F401
    IN_COLAB = True
except Exception:
    IN_COLAB = False
    class _FilesShim:
        def upload(self):
            print("Local mode: place files in the expected folders; no upload dialog.")
            return {}
        def download(self, *args, **kwargs):
            print("Local mode: file saved on disk; no download dialog.")
    files = _FilesShim()

# Step 2: Integrate MoGE init into on-the-fly 3DGS

This notebook loads the dense initialization (PLY) produced in Step 1 and starts the on-the-fly incremental optimizer from that prior, bypassing the original multi-view bootstrap.

- Place your init PLY at the workspace root (default name `3dgs_Moge.ply`) or edit the path variable.
- It creates `on-the-fly-nvs/results/scene/point_cloud/point_cloud.ply` so the training picks it up as the starting scene.
- Works on Colab and locally; paths are dynamically set.

In [ ]:
# Portable paths for Colab vs local
import os
from pathlib import Path


def is_colab():
    return "COLAB_RELEASE_TAG" in os.environ or "COLAB_GPU" in os.environ


ROOT = Path.cwd()
REPO = "/content/on-the-fly-nvs" if is_colab() else str((ROOT / "on-the-fly-nvs").resolve())

# Common dirs
SCENE_DIR = f"{REPO}/data/my_scene"
MODEL_DIR = f"{REPO}/results/scene"

# PLY init path: expects a file at workspace root named '3dgs_Moge.ply' by default
DEFAULT_PLY_NAME = "3dgs_Moge.ply"
PLY_PATH = str((ROOT / DEFAULT_PLY_NAME).resolve()) if (ROOT / DEFAULT_PLY_NAME).exists() else ""

print("REPO:", REPO)
print("MODEL_DIR:", MODEL_DIR)
print("SCENE_DIR:", SCENE_DIR)
print("PLY_PATH:", PLY_PATH or "(not found)")

In [ ]:
#@title 1) Clone and install on-the-fly-nvs + dependencies
!git clone --recursive https://github.com/graphdeco-inria/on-the-fly-nvs.git
%cd on-the-fly-nvs

!pip install -q -r requirements.txt

import torch
print("Torch:", torch.__version__)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)
assert device=="cuda", "GPU required for smooth training."


Cloning into 'on-the-fly-nvs'...
remote: Enumerating objects: 808, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 808 (delta 22), reused 14 (delta 13), pack-reused 773 (from 1)
Receiving objects: 100% (808/808), 5.20 MiB | 10.42 MiB/s, done.
Resolving deltas: 100% (259/259), done.
Submodule 'submodules/Depth-Anything-V2' (https://github.com/DepthAnything/Depth-Anything-V2.git) registered for path 'submodules/Depth-Anything-V2'
Submodule 'submodules/fused-ssim' (https://github.com/rahul-goel/fused-ssim) registered for path 'submodules/fused-ssim'
Submodule 'submodules/graphdecoviewer' (https://github.com/graphdeco-inria/graphdecoviewer.git) registered for path 'submodules/graphdecoviewer'
Cloning into '/content/on-the-fly-nvs/submodules/Depth-Anything-V2'...
remote: Enumerating objects: 142, done.        
remote: Total 142 (delta 0), reused 0 (delta 0), pack-reused 142 (from 1)        
Receiving objects: 100% (142/142),

In [ ]:
#@title 2) Prepare scene paths (images folder, model dir)
import os, zipfile, json, io, shutil, re

# Reuse portable paths computed above
SOURCE_ROOT = SCENE_DIR
IMAGES_DIR  = os.path.join(SOURCE_ROOT, "images")
os.makedirs(IMAGES_DIR, exist_ok=True)
os.makedirs(MODEL_DIR,  exist_ok=True)

print("Scene dir:", SOURCE_ROOT)
print("Images dir:", IMAGES_DIR)
print("Model dir:", MODEL_DIR)
print("Init PLY:", PLY_PATH or "(not found)")


In [ ]:
#@title 3A) Utilitaires pour lire un PLY 3DGS (MoGE-2→3DGS)
import numpy as np
from plyfile import PlyData

def read_3dgs_ply(path):
    ply = PlyData.read(path)
    v = ply['vertex'].data
    names = v.dtype.names
    def get(name, default=None, shape=None):
        if name in names:
            arr = np.asarray(v[name], dtype=np.float32)
            if shape and arr.ndim==1 and shape>1: arr = arr.reshape(-1,1)
            return arr
        return default
    xyz = np.stack([v['x'], v['y'], v['z']], axis=1).astype(np.float32)
    fdc = np.stack([get('f_dc_0', np.zeros(len(v))),
                    get('f_dc_1', np.zeros(len(v))),
                    get('f_dc_2', np.zeros(len(v)))], axis=1).astype(np.float32)
    opa = get('opacity', np.ones(len(v), np.float32)).astype(np.float32)
    s   = np.stack([get('scale_0', np.ones(len(v))),
                    get('scale_1', np.ones(len(v))),
                    get('scale_2', np.ones(len(v)))], axis=1).astype(np.float32)
    r   = np.stack([get('rot_0', np.ones(len(v))),
                    get('rot_1', np.zeros(len(v))),
                    get('rot_2', np.zeros(len(v))),
                    get('rot_3', np.zeros(len(v)))], axis=1).astype(np.float32)
    return dict(xyz=xyz, features_dc=fdc, opacity=opa, scales=s, rots=r, count=len(v))

test = read_3dgs_ply(PLY_PATH) if PLY_PATH else None
print("PLY stats:", None if test is None else {k: (v.shape if hasattr(v,'shape') else len(v)) for k,v in test.items() if k!='count'})


PLY stats: {'xyz': (1254528, 3), 'features_dc': (1254528, 3), 'opacity': (1254528,), 'scales': (1254528, 3), 'rots': (1254528, 4)}


In [ ]:
#@title 3B) Copy init PLY (MoGE) and use repo hook
import sys, pathlib, textwrap, glob, os, shutil

# Copy the init 3DGS PLY to the conventional location so train.py picks it up
POINTCLOUD_DIR = os.path.join(MODEL_DIR, "point_cloud")
os.makedirs(POINTCLOUD_DIR, exist_ok=True)
PC_PLY = os.path.join(POINTCLOUD_DIR, "point_cloud.ply")
if PLY_PATH:
    shutil.copy(PLY_PATH, PC_PLY)
    print("Init PLY copied to:", PC_PLY)
else:
    print("No PLY found; training will fall back to geometric bootstrap.")

# Do not rewrite the hook here; rely on init_from_ply_hook.py committed in the repo
HOOK_PATH = os.path.abspath("init_from_ply_hook.py")
if os.path.exists(HOOK_PATH):
    print("Using repo hook:", HOOK_PATH)
else:
    print("WARNING: hook not found at:", HOOK_PATH)
    print("Please keep init_from_ply_hook.py at the repo root to enable starting from existing PLY.")


Init PLY copié vers: results/my_scene/point_cloud/point_cloud.ply
Hook écrit: /content/on-the-fly-nvs/init_from_ply_hook.py


In [ ]:
#@title 4) Run on-the-fly incremental optimization
import subprocess, shlex, sys, os

# Expose the hook via PYTHONPATH so train.py can import it
env = os.environ.copy()
prev_pp = env.get("PYTHONPATH", "")
extra = os.getcwd()
env["PYTHONPATH"] = os.pathsep.join([p for p in [prev_pp, extra] if p])

# Base command (see README)
cmd = f"python train.py -s {SOURCE_ROOT} -m {MODEL_DIR} --viewer_mode none --downsampling 2.5 --save_every 100"
print("RUN:", cmd)
ret = subprocess.call(shlex.split(cmd), env=env)
print("Exit code:", ret)


RUN: python train.py -s data/my_scene -m results/my_scene --viewer_mode none --downsampling 2.8 --save_every 100 ----save_at_finetune_epoch 10
Exit code: 2


In [ ]:
#@title 5A) Render a trajectory: reuse optimized COLMAP poses
import os, glob, subprocess, shlex

COLMAP_DIR = os.path.join(MODEL_DIR, "colmap")
assert os.path.isdir(COLMAP_DIR), "No 'colmap/' found (incomplete training?)."

OUT_RENDER = os.path.join(MODEL_DIR, "renders")
os.makedirs(OUT_RENDER, exist_ok=True)

cmd = f"python scripts/render_path.py -m {MODEL_DIR} --render_path {COLMAP_DIR} --out_dir {OUT_RENDER}"
print("RUN:", cmd)
subprocess.call(shlex.split(cmd))
print("Renders saved to:", OUT_RENDER)


RUN: python scripts/render_path.py -m results/my_scene --render_path results/my_scene/colmap --out_dir results/my_scene/renders
Rendus dans: results/my_scene/renders
